In [1]:
import numpy as np
# from google.colab import drive

# This will prompt for authorization.
# drive.mount('/content/drive')
import sys
import tensorflow as tf

In [2]:
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

True

In [10]:
# !ls

from pathlib import Path
import glob

In [11]:
# path = '/content/drive/My Drive/Colab Notebooks/Music/ADEHack2018LSTM/queen/'
#path = 'C:\\Users\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen\\'
#path = 'Documents//GitHub//tensorflow_1.X_playground//time_series//models//queen//midi//'

path = Path("Documents/GitHub/tensorflow_1.X_playground/time_series/models/queen/")
path = 'C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen\\'

In [12]:
#os.listdir(path)

In [13]:
sys.path


['C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen',
 'C:\\Users\\woooh\\Miniconda3\\envs\\tf_gpu\\python37.zip',
 'C:\\Users\\woooh\\Miniconda3\\envs\\tf_gpu\\DLLs',
 'C:\\Users\\woooh\\Miniconda3\\envs\\tf_gpu\\lib',
 'C:\\Users\\woooh\\Miniconda3\\envs\\tf_gpu',
 '',
 'C:\\Users\\woooh\\Miniconda3\\envs\\tf_gpu\\lib\\site-packages',
 'C:\\Users\\woooh\\Miniconda3\\envs\\tf_gpu\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\woooh\\.ipython']

In [14]:
# !cd '/content/drive/My Drive/Colab Notebooks/Music/ADEHack2018LSTM/queen'
print(glob.glob(path+"midi/*.mid"))


['C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen\\midi\\Queen_-_Another_One_Bites_the_Dust.mid', 'C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen\\midi\\Queen_-_Bicycle_Race.mid', 'C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen\\midi\\Queen_-_Bohemian_Rhapsody.mid', 'C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen\\midi\\Queen_-_Crazy_Little_Thing_Called_Love.mid', 'C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen\\midi\\Queen_-_Father_to_Son.mid', 'C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen\\midi\\Queen_-_Fat_Bottom_Girls.mid', 'C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen\\midi\\Queen_-_i_want_to_break_free.mid', 'C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\ti

In [15]:
#!pip install music21

In [16]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.python.keras import utils
from tensorflow.keras.callbacks import ModelCheckpoint


In [17]:

def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []
    
    print(glob.glob(path+"midi\\*.mid"))

    for file in glob.glob(path+"midi\\*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            print('no instruments')
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open(path+'data\\notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    #print(notes[:100])
    return notes

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = utils.to_categorical(network_output)

    return (network_input, network_output)



In [22]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = path+"checkpoints\\ckpt.hdf5"#"checkpoints/new-weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    #model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)
    model.fit(network_input, network_output, epochs=20, batch_size=32, callbacks=callbacks_list)


In [24]:
#chkpt = path+"ckpt"#"checkpoints/new-weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"


In [ ]:
import glob
import os

list_of_files = glob.glob(path+'checkpoints/*.hdf5') # gets latest checkpoint
chkpt = max(list_of_files, key=os.path.getctime)

#print('restoring from'.format(chkpt))

def create_network_with_chkpt(network_input, n_vocab):
    """ create the structure of the neural network """
    
    model = Sequential()
    model.add(LSTM(
        250,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.2))
    model.add(LSTM(300, return_sequences=True))
    model.add(Dropout(0.4))
    model.add(LSTM(300))
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights(chkpt)

    return model
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = np.random.randint(10,100)

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = utils.to_categorical(network_output)

    return (network_input, network_output)
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))
    #print(n_vocab)

    network_input, network_output = prepare_sequences(notes, n_vocab)
    #print(notes)
    #print(n_vocab)

    model = create_network_with_chkpt(network_input, n_vocab)

    train(model, network_input, network_output)
if __name__ == '__main__':
    train_network()

['C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen\\midi\\Queen_-_Another_One_Bites_the_Dust.mid', 'C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen\\midi\\Queen_-_Bicycle_Race.mid', 'C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen\\midi\\Queen_-_Bohemian_Rhapsody.mid', 'C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen\\midi\\Queen_-_Crazy_Little_Thing_Called_Love.mid', 'C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen\\midi\\Queen_-_Father_to_Son.mid', 'C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen\\midi\\Queen_-_Fat_Bottom_Girls.mid', 'C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\time_series\\models\\queen\\midi\\Queen_-_i_want_to_break_free.mid', 'C:\\Users\\woooh\\Documents\\GitHub\\tensorflow_1.X_playground\\ti

In [ ]:
chkpt

In [ ]:
chkpt